In [19]:
from modeling.gen6.api import symbolic, acos
from modeling.gen7.api7 import MFunctionalSet, MFunctionalSetLeaf, FunctionSympy
from engine.torchdata import load_vals, print_formatted_table
import numpy as np
from scipy import optimize

In [199]:
h, hkm, a, T, g, d, r = symbolic('h', 'hkm', 'a', 'T', 'g', 'd', 'r')
R, mu = 6371e3, 3.986e14
HMult = MFunctionalSetLeaf(
    h==hkm*1000
)
Orbit = MFunctionalSetLeaf(
    a==h+R, 
    T==2*np.pi*(a**3/mu)**0.5, 
    g==1/np.pi*acos(R/a), 
    d==g+0.5, 
    r==(h**2+2*R*h)**0.5
) # by default configured to elimination of state variables

m_A, P_c, P_T, E_b, m_b = symbolic('m_A', 'P_c', 'P_T', 'E_b', 'm_b')
eta_A, rho_A, rho_b, P_l, A, Q = 0.3, 10, 0.002e-3, 12, 0.05, 1365
Power = MFunctionalSetLeaf(
    m_A==rho_A*A, 
    P_c==d*A*Q*eta_A, 
    P_T==P_c-P_l, 
    E_b==P_c*T/d, 
    m_b==rho_b*E_b
)

X_r, D_p, D, m_p = symbolic('X_r', 'D_p', 'D', 'm_p')
rho_p, l_v, B, N = 2, 500e-9, 8, 2000
Payload = MFunctionalSetLeaf(
    D_p==1.22*l_v*h/X_r, 
    D==2*np.pi*R*B*N/X_r, 
    m_p==rho_p*D_p**1.5
)

b, l_c, D_T, m_T, EN, G_r = symbolic('b', 'l_c', 'D_T', 'm_T', 'EN', 'G_r')
rho_T, G_T, D_r, eta, T_s, k, c, F = 0.2, 16.5, 5.3, 0.55, 135, 1.38064852e-23, 2.99e8, 2.2e9
dBtoLinear = lambda db: 10**(db/10)
L = dBtoLinear(1+8.5+0.3+0.1)
Comms = MFunctionalSetLeaf(
    b==D/T, 
    l_c==c/F,
    D_T==l_c*(G_T/eta)**0.5/np.pi,
    m_T==rho_T*D_T**1.5,
    G_r==eta*(np.pi*D_r/l_c)**2,
    EN==P_T*G_r*G_T/(L*k*T_s*b)*(l_c/(4*np.pi*r))**2,
)

mt, m_s = symbolic('m_t', 'm_s')
eta_S, m_pr = 0.2, 0.5
Struct = MFunctionalSetLeaf(
    m_s == eta_S*mt
)
Mass = MFunctionalSetLeaf(
    mt == m_A + m_b + m_p + m_T + m_pr + m_s 
)

MDA = MFunctionalSet(HMult, Orbit, Power, Payload, Comms)
FPF = MDA.subsetof(5 <= X_r, X_r <= 100,
                   300e3 <= h ).minimize(m_A + m_b + m_p + m_T)

In [149]:
MDAp = MFunctionalSet(Orbit, Power, Comms).config(elim=[Orbit, Power, Comms])
_,_,_,_, indices = MDAp.gather_sets()
f = MDAp.build()
x0 = {"h": 400e3, "D": 0.1}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

d    EN       r      P_T   a      h      m_A m_T   T        D_T   D   g    G_r      b        E_b    m_b   P_c    l_c  
0.61 4.51e+12 2.29e6 0.489 6.77e6 4.00e5 0.5 0.023 5544.858 0.237 0.1 0.11 8255.002 1.80e-05 1.14e5 0.227 12.489 0.136


In [150]:
_,_,_,_, indices = Power.gather_sets()
f = Power.build()
x0 = {"d": 0.6, "T": 6000}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

T    E_b    d   m_b   P_c    P_T   m_A
6000 1.23e5 0.6 0.246 12.285 0.285 0.5


In [151]:
_,_,_,_, indices = Orbit.gather_sets()
f = Orbit.build()
x0 = {"h": 400e3}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

r      T        d    a      h      g   
2.29e6 5544.858 0.61 6.77e6 4.00e5 0.11


In [200]:
SPF_MDF1 = FPF.config(elim=[HMult, Orbit, Power, Payload, Comms])
sets, ineq_constraints_merged, eq_constraints, obj, indices = SPF_MDF1.gather_sets()
f = SPF_MDF1.build()
x0 = {"hkm": 400, "X_r": 1}
xidxed = load_vals(x0, indices, isdict=True)
f0 = f.analysis(xidxed).detach().clone()
idxrev = {var.item():key for key,var in indices.items()}
valdict = {val:f0[key].item() for key,val in idxrev.items()}
# print_formatted_table([f.analysis(xidxed)], indices, idxrev)

In [202]:
obj_function, dobj, xguess, constraints  = SPF_MDF1.build_opt(x0=f0)

In [203]:
dobj(xguess), constraints[0]['fun'](xguess)

(array([ 0.00095426, -0.36158133]),
 tensor([-4.0000e+00,  9.9000e+01,  1.0000e+05], dtype=torch.float64))

In [208]:
xsol = optimize.minimize(obj_function, xguess, jac=dobj, constraints=constraints, method='SLSQP', 
                         options={'ftol': 1e-9})

In [209]:
xsol.x

array([300., 100.])

In [211]:
xidxed = load_vals({'hkm':xsol.x[0], 'X_r':xsol.x[1]}, 
                   indices, isdict=True)
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

d     EN     r      P_T   a      h      m_A m_T   X_r T        D_p      D_T   D      g     G_r      hkm b      E_b    m_b   P_c    l_c   m_p     
0.596 37.757 1.98e6 0.199 6.67e6 3.00e5 0.5 0.023 100 5422.476 1.83e-03 0.237 6.40e9 0.096 8255.002 300 1.18e6 1.11e5 0.222 12.199 0.136 1.57e-04


In [212]:
xsol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7452754334567794
       x: [ 3.000e+02  1.000e+02]
     nit: 27
     jac: [ 5.071e-05 -2.349e-06]
    nfev: 27
    njev: 27

In [77]:
print_formatted_table([f0], indices, idxrev)

T        X_r      d    D_p   D        g    r      E_b    P_T   m_b   h      P_c    m_p   a      m_A
5544.858 2085.041 0.61 0.244 6.40e+11 0.11 2.29e6 1.14e5 0.489 0.227 4.00e5 12.489 0.241 6.77e6 0.5


In [ ]:
SPF_IDF = FPF.config(parallel=[Orbit, Power, Payload])
SPF_AAO = FPF.config(residuals=Orbit.supersets+Power.supersets+Payload.supersets)
SPF_MDF1 = FPF.config(elim=[Orbit, Power, Payload])
SPF_MDF2 = FPF.config(elim=[MDA.config(parallel=[Orbit, Power, Payload])])

In [132]:
# HMult = MFunctionalSetLeaf(
#     h==hkm*1000
# )